In [1]:
import warnings
# External Imports
from IPython.display import display, clear_output
from tkinter import Tk,filedialog
from ipyleaflet import GeoJSON
# import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox, Layout, DatePicker, HTML, RadioButtons, SelectMultiple, Output, Checkbox
import ipywidgets
import os
import datetime
import glob
import json
import pandas as pd
import geopandas as gpd
from google.auth import exceptions as google_auth_exceptions
import ee
# New imports for coastsat shoreline extraction
import numpy as np
# Local Imports 
from CoastSeg import download_roi, file_functions, zoo_model_module, coastseg_map_class
from CoastSeg.CoastSat.coastsat import SDS_shoreline,SDS_download
# suppress tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
root = Tk()
remove_style=dict(button_color = 'red')
load_style=dict(button_color = '#69add1')
action_style=dict(button_color = '#ae3cf0')
save_style=dict(button_color = '#50bf8f')

Version:  2.9.1
Eager mode:  True
Version:  2.9.1
Eager mode:  True
GPU name:  []
Num GPUs Available:  0


## Authenticate and Initialize with Google Earth Engine (GEE)
- Run this cell to initialize with GEE which will allow you to download remote sensing data from GEE.
### First Time Users
- In order to use Google Earth Engine (GEE) you will need to sign up to request access to use Google Earth Engine.https://signup.earthengine.google.com. You will only need to do this once and it takes only a day to get your account verified.
### How `ee.Authenticate()` works
- In order to initialize with GEE you will need an authorization token with is obtained by running `ee.Authenticate()`.This token lasts 7 days and during those 7 days you will not need to authenticate with google earth engine with an access code. Once the 7 days are up you will need to reauthenticate to use GEE again.


In [2]:
try:
    ee.Initialize()
except google_auth_exceptions.RefreshError as exception:
    print("Please authenticate with Google:\n")
    ee.Authenticate()
    ee.Initialize()

In [3]:
# Date Widgets
start_date=DatePicker(
    description='Start Date',
    value=datetime.date(2018, 12, 1),
    disabled=False,
)
end_date=DatePicker(
    description='End Date',
    value=datetime.date(2019, 3, 1), #2019, 1, 1
    disabled=False,
)
date_instr=HTML(
    value="<b>Pick a date:</b>",
    layout=Layout(padding='10px')
)
dates_box=HBox([start_date,end_date])
dates_vbox=VBox([date_instr,dates_box])


collection_instr=HTML(
    value="<b>Pick a collection</b>\
        <br> C01: LandSat Collection 1 \
        <br> C02: LandSat Collection 2  [2022/01/01 and beyond] \
        <br>      - Landsat 9 is only available in C02",
    layout=Layout(padding='10px')
)
collection_radio=RadioButtons(
    options=['C01', 'C02'],
    value='C01',
    description='Collection:',
    disabled=False
)
collection_vbox=VBox([collection_instr,collection_radio])

sat_instr=HTML(
    value="<b>Pick multiple satellites by holding the control key:</b> \
        <br> - images prior to 2022/01/01 will be downloaded from Collection 1 \
        <br> - images after 2022/01/01 will be automatically downloaded from Collection 2 \
        <br> C01: LandSat Collection 1 : All dates before 2022/01/01\
        <br> C02: LandSat Collection 2 : All dates after 2022/01/01 \
        <br> - Landsat 9 is only available in C02",
    layout=Layout(padding='10px')
)


satellite_selection=SelectMultiple(
    options=['L5', 'L7', 'L8', 'S2'],
    value=['L8'],
    description='Satellites',
    disabled=False
)
sat_vbox = VBox([sat_instr,satellite_selection])

def handle_collection_change(change):
    if change['new'] == 'C02':
        satellite_selection.options=['L5', 'L7', 'L8', 'L9', 'S2']
    else:
        satellite_selection.options=['L5', 'L7', 'L8', 'S2']

collection_radio.observe(handle_collection_change,"value")

controls_vbox=VBox([dates_vbox, collection_vbox, sat_vbox])
controls_vbox

In [4]:
# CoastSat Download Variables
# ------------------------------
# Save the satellites selected to sat_list
if satellite_selection.value:
    sat_list = list(satellite_selection.value)
elif not satellite_selection.value:
    print("ERROR: You must select at least one satellite first")
# Save the dates selected by the user as well as the selected collection
dates = [str(start_date.value),str(end_date.value)]
collection = collection_radio.value
print(f"dates: {dates}")
print(f"collection: {collection}")
print(f"sat_list: {sat_list}")

dates: ['2018-12-01', '2019-03-01']
collection: C01
sat_list: ['L8']


## Settings for CoastSat
Modify any of the following setting to determine how data is downloaded from CoastSat

In [5]:
pre_process_settings = { 
    # general parameters:
    'cloud_thresh': 0.5,        # threshold on maximum cloud cover
     'dist_clouds': 300,        # ditance around clouds where shoreline can't be mapped
    'output_epsg': 3857,        # epsg code of spatial reference system desired for the output   
    # quality control:
    'check_detection': True,    # if True, shows each shoreline detection to the user for validation
    'adjust_detection': False,  # if True, allows user to adjust the postion of each shoreline by changing the threhold
    'save_figure': True,        # if True, saves a figure showing the mapped shoreline for each image
    # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
    'min_beach_area': 4500,     # minimum area (in metres^2) for an object to be labelled as a beach
    'buffer_size': 150,         # radius (in metres) of the buffer around sandy pixels considered in the shoreline detection
    'min_length_sl': 200,       # minimum length (in metres) of shoreline perimeter to be valid
    'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
    'sand_color': 'default',    # 'default', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
    'pan_off':'False',          # if True, no pan-sharpening is performed on Landsat 7,8 and 9 imagery
}

# Create the Map
- Create the map object here

In [6]:
coastseg_map=coastseg_map_class.CoastSeg_Map()

# How to Use The Map
---
1. Use the rectangle tool to draw a bounding box along the coastline.
2. Click `Generate ROI` to create ROI rectangles along the coastline in the bounding box. This may take some time.
- You should see a coastline appear in yellow and some rectangles along it.
3. Click 1 or more ROIs then click `Save ROI` to save these ROIs for downloading later
4. You're ready to download the ROIs. Run the next block of code.

In [7]:
# Controls size of ROIs generated on map
small_roi_size = 3500
large_roi_size = 4000
fishnet_sizes={'small':small_roi_size,'large':large_roi_size}

# Output wdiget used to print messages and exceptions created by download progress
download_view = Output(layout={'border': '1px solid black'})
# Output wdiget used to print messages and exceptions created by CoastSeg_Map
debug_view = Output(layout={'border': '1px solid black'})


# load buttons
transects_button = Button(description="Load Transects",style=load_style)
shoreline_button = Button(description="Load Shoreline",style=load_style)
load_rois_button = Button(description="Load rois from file",style=load_style)
load_bbox_button = Button(description="Load bbox from file",style=load_style)

# Save and Generate buttons
gen_button =Button(description="Generate ROI",style=action_style)
save_button = Button(description="Save ROI",style=save_style)
save_bbox_button = Button(description="Save Bbox",style=save_style)
download_button = Button(description="Download ROIs",style=action_style)


# Remove buttons
clear_debug_button = Button(description="Clear TextBox")
clear_debug_button.style.button_color = '#a3adac'

remove_all_button = Button(description="Remove all",style=remove_style)
remove_transects_button = Button(description="Remove transects",style=remove_style)
remove_bbox_button = Button(description="Remove bbox",style=remove_style)
remove_coastline_button = Button(description="Remove coastline",style=remove_style)
remove_rois_button = Button(description="Remove ROIs",style=remove_style)

# widgets
instr_create_roi=HTML(
    value="<h2><b>Generate ROIs</b></h2> \
        Use the two sliders to control the size of the ROIs generated.<br>\
            <li> No Overlap: Set small slider to 0 and large slider to desired ROI size.</li>\
            <li>Overlap: Set small slider to a value and large slider to desired ROI size.</li>\
            <b>How it Works</b><br> \
            Two fishnets (grids) are created out of ROIs (squares) and placed <br> \
            within the bounding box.You control the size of the individual ROIs <br> \
            that compose the small and large fishnets.<br>\
            The ROIs are measured in meters squared.",layout=Layout(margin='0px 5px 0px 0px'))

instr_save_roi=HTML(
    value="<h2><b>Save ROIs</b></h2> \
        You must click save ROI before you can download ROIs\
        <br> Use the save buttons to save the ROIs you selected <br>\
           <li> Save BBox: Save the bounding box to a geojson file called 'bbox.geojson'</li>\
            <li> Save ROIs: Save the selected ROI to a geojson file called 'rois.geojson'</li>\
            ",layout=Layout(margin='0px 5px 0px 5px')) #top right bottom left

instr_download_roi=HTML(
    value="<h2><b>Download ROIs</b></h2> \
         <b> You must click 'Save ROI' before you can download ROIs</b> \
          </br> Scroll past the map to see the download progress \
            <li> The data you downloaded will be in the 'data' folder</li> \
            ",layout=Layout(margin='0px 0px 0px 5px'))

instr_load_btns=HTML(
    value="<h2><b>Load ROIs/BBox</b></h2>\
         You can upload ROIs or Bbox geojson file.\
            <li> Load BBox: Load bounding box from geojson file (ex. 'bbox.geojson')</li>\
           <li> Load ROIs: Load ROIs from geojson file (ex. 'rois.geojson')</li>\
            ",layout=Layout(margin='0px 5px 0px 5px')) #top right bottom left

slider_style = {'description_width': 'initial'}
small_fishnet_slider=ipywidgets.IntSlider(
    value=small_roi_size,
    min=0,
    max=10000,
    step=100,
    description='Small Fishnet:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style=slider_style
)

large_fishnet_slider=ipywidgets.IntSlider(
    value=large_roi_size,
    min=1000,
    max=10000,
    step=100,
    description='Large Fishnet:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style=slider_style
)

def handle_small_slider_change(change):
    fishnet_sizes['small']=change['new']
def handle_large_slider_change(change):
    fishnet_sizes['large']=change['new']

@download_view.capture(clear_output=True)
def download_button_clicked(btn):
    
    download_view.clear_output()
    debug_view.clear_output()
    if coastseg_map.selected_ROI:
        coastseg_map.m.default_style = {'cursor': 'wait'}
        debug_view.append_stdout("Scroll down past map to see download progress.")
        try:
            download_button_clicked.disabled=True
            download_roi.download_imagery(coastseg_map.selected_ROI,pre_process_settings,dates,sat_list,collection)
        except google_auth_exceptions.RefreshError as exception:
            print(exception)
            print("Please authenticate with Google using the cell above: \n  'Authenticate and Initialize with Google Earth Engine (GEE)'")
    else:
        debug_view.append_stdout("No ROIs were selected. \nPlease select at least one ROI and click 'Save ROI' to save these ROI for download.")
    download_button_clicked.disabled=False
    coastseg_map.m.default_style = {'cursor': 'default'}

@debug_view.capture(clear_output=True)
def on_shoreline_button_clicked(b):
    if coastseg_map.shapes_list == [] :
        print("Draw a bounding box on the coast first, then click Load Transects.")
    else:
        coastseg_map.m.default_style = {'cursor': 'wait'}
        debug_view.clear_output(wait=True)
        print("Loading shoreline please wait.") 
        # Add the transects to the map
        coastseg_map.load_shoreline_on_map()
        print("Shoreline loaded.")
        coastseg_map.m.default_style = {'cursor': 'default'}
        
@debug_view.capture(clear_output=True)
def on_load_rois_clicked(button):
    # Prompt the user to select a directory of images                              
    root.withdraw()                                        # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
    root.filename =  filedialog.askopenfilename(initialdir = os.getcwd(),
                                                filetypes=[('geojson','*.geojson')],
                                                title = "Select a geojson file containing rois")
    # Save the filename as an attribute of the button
    if root.filename:
        roi_file= root.filename
        rois = gpd.read_file(roi_file)
        # style fishnet and convert to dictionary to be added to map
        rois_dict = coastseg_map.style_rois(rois)
        # Save the styled fishnet to data for interactivity to be added later
        coastseg_map.data = rois_dict
        # Add the Clickable ROIs to the map
        coastseg_map.add_geojson_layer_to_map()
        
        print(f"Loaded the rois from the file :\n{roi_file} ")
    else:
        raise Exception("You must select a valid geojson file first!")

@debug_view.capture(clear_output=True)
def on_load_bbox_clicked(button):
    # Prompt the user to select a directory of images                              
    root.withdraw()                                        # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
    root.filename =  filedialog.askopenfilename(initialdir = os.getcwd(),
                                                filetypes=[('geojson','*.geojson')],
                                                title = "Select a geojson file containing bbox")
    # Save the filename as an attribute of the button
    if root.filename:
        bbox_file= root.filename
        bbox_geodf = gpd.read_file(bbox_file)
        bbox_geojson = bbox_geodf.to_json()
        bbox_dict = json.loads(bbox_geojson)
        coastseg_map.shapes_list.append(bbox_dict['features'][0]['geometry'])
        bbox_layer = GeoJSON(
            data=bbox_dict,
            name="Bbox",
            style={
                'color': '#75b671',
                'fill_color': '#75b671',
                'opacity': 1,
                'fillOpacity': 0.2,
                'weight': 4},
        )
        coastseg_map.m.add_layer(bbox_layer)
        
        print(f"Loaded the rois from the file :\n{bbox_file} ")
    else:
        raise Exception("You must select a valid geojson file first!")



@debug_view.capture(clear_output=True)
def on_transects_button_clicked(b):
    if coastseg_map.shapes_list == [] :
        print("Draw a bounding box on the coast first, then click Load Transects.")
    else:
        debug_view.clear_output(wait=True)
        coastseg_map.m.default_style = {'cursor': 'wait'}
        print("Loading transects please wait.") 
        # Add the transects to the map
        coastseg_map.load_transects_on_map()
        print("Transects Loaded.")
        coastseg_map.m.default_style = {'cursor': 'default'}

        
@debug_view.capture(clear_output=True)
def on_gen_button_clicked(b):
    if coastseg_map.shapes_list == [] :
        print("Draw a bounding box on the coast first, then click Generate ROI.")
    else:
        debug_view.clear_output(wait=True)
        coastseg_map.m.default_style = {'cursor': 'wait'}
        print("Generating ROIs please wait.") 
        # Generate ROIs along the coastline within the bounding box
        coastseg_map.generate_ROIS_fishnet(fishnet_sizes['large'],fishnet_sizes['small'])
        # coastseg_map.generate_ROIS_fishnet(5000,0)
        # debug_view.clear_output(wait=True)
        # Add the Clickable ROIs to the map
        coastseg_map.add_geojson_layer_to_map()
        print("ROIs generated. Please Select at least one ROI and click Save ROI.")
        coastseg_map.m.default_style = {'cursor': 'default'}
        
@debug_view.capture(clear_output=True)
def on_save_bbox_button_clicked(b):
    if coastseg_map.shapes_list != [] :
        debug_view.clear_output(wait=True)
        # Save selected bbox to a geojson file
        coastseg_map.save_bbox_to_file() 
        debug_view.clear_output(wait=True)
        print("BBox have been saved. Saved to bbox.geojson")
    else:
         print("Draw a bounding box on the coast first")

        
@debug_view.capture(clear_output=True)
def on_save_button_clicked(b):
    if coastseg_map.selected_set:
        if len(coastseg_map.selected_set) == 0:
             print("Must select at least 1 ROI first before you can save ROIs.")
        else:
            debug_view.clear_output(wait=True)
            coastseg_map.save_roi_fishnet("fishnet_rois.geojson")
            print("Saving ROIs")
            debug_view.clear_output(wait=True)
            print("ROIs have been saved. Now click Download ROI to download the ROIs using CoastSat")
    else:
        print("No ROIs were selected.")


def remove_all_from_map(b):
    coastseg_map.remove_all()
def remove_transects(b):
    coastseg_map.remove_transects()
def remove_bbox_from_map(b):
    coastseg_map.remove_bbox()
def remove_coastline_from_map(b):
    coastseg_map.remove_shoreline()
def remove_all_rois_from_map(b):
    coastseg_map.remove_all_rois()
def clear_debug_view(b):
    debug_view.clear_output()
    download_view.clear_output()

# widget handlers
small_fishnet_slider.observe(handle_small_slider_change,'value')
large_fishnet_slider.observe(handle_large_slider_change,'value')

# button handlers
download_button.on_click(download_button_clicked)
        
transects_button.on_click(on_transects_button_clicked)    
shoreline_button.on_click(on_shoreline_button_clicked) 
load_rois_button.on_click(on_load_rois_clicked)
load_bbox_button.on_click(on_load_bbox_clicked)

gen_button.on_click(on_gen_button_clicked)
save_button.on_click(on_save_button_clicked)
save_bbox_button.on_click(on_save_bbox_button_clicked)
clear_debug_button.on_click(clear_debug_view)


remove_all_button.on_click(remove_all_from_map)
remove_transects_button.on_click(remove_transects)
remove_bbox_button.on_click(remove_bbox_from_map)
remove_coastline_button.on_click(remove_coastline_from_map)
remove_rois_button.on_click(remove_all_rois_from_map)

save_vbox = VBox([instr_save_roi,save_button,save_bbox_button, instr_load_btns, load_rois_button, load_bbox_button])
download_vbox = VBox([instr_download_roi,download_button])

slider_v_box = VBox([small_fishnet_slider, large_fishnet_slider])
slider_btn_box = HBox([slider_v_box, gen_button])
roi_controls_box = VBox([instr_create_roi, slider_btn_box],layout=Layout(margin='0px 5px 5px 0px'))

action_buttons =  HBox([clear_debug_button])
load_buttons = HBox([transects_button, shoreline_button])
erase_buttons = HBox([remove_all_button, remove_transects_button, remove_bbox_button, remove_coastline_button, remove_rois_button])


row_1 = HBox([roi_controls_box,save_vbox,download_vbox])
row_2 = HBox([load_buttons])
row_3 = HBox([erase_buttons])
row_4 = HBox([clear_debug_button, debug_view])
row_5 = HBox([coastseg_map.m])
row_6 = HBox([download_view])

display(row_1,row_2, row_3, row_4, row_5, row_6)

## Download the Selected ROIs
- Download the satellite imagery for the ROIs using CoastSat.

⚠️ Make sure you have clicked `Save ROI` before running the following code.
 
### Parameters for download_imagery()
1. `coastseg_map.selected_ROI` : geojson for the ROI clicked
2. `pre_process_settings`: settings for downloading data using CoastSat
3. `dates` : date range to download the imagery
4. `sat_list` : list of satellites to download imagery from
5. `collection` : LandSat Collection 1 (`C01`) or Collection 2 (`C02`)

In [9]:
model_dict={'sample_direc':None,'use_GPU':False,'use_CRF':False, 'implementation':'ENSEMBLE','model_type':'landsat_6229071'}

model_view = Output(layout={'border': '1px solid black'})
run_model_view = Output(layout={'border': '1px solid black'})
GPU_view = Output()

run_model_button = Button(description="Run Model",style=action_style)
use_data_button = Button(description="Use Data Button",style=load_style)
use_select_images_button = Button(description="Select Your Images",style=load_style)
open_results_button = Button(description="Open Model Results",style=load_style)

model_implementation = RadioButtons(
    options=['ENSEMBLE', 'BEST'],
   value='ENSEMBLE',
    description='Select:',
    disabled=False
)

model_dropdown = RadioButtons(
    options= ['landsat_6229071', 'landsat_6230083','SWED-RGB_6824384','SWED-MNDWI_6824342','coast-train-RGB_6950479', 'S2-water-SWED_6950474'],
    value='landsat_6229071',
    description='Select Model:',
    disabled=False,
)


GPU_checkbox = Checkbox(
    value=False,
    description='Use GPU?',
    disabled=False,
    indent=False
)

CRF_checkbox = Checkbox(
    value=False,
    description='Use CRF post-processing',
    disabled=False,
    indent=False
)


def handle_model_type(change):
    model_dict["model_type"]=change['new']


def handle_model_implementation(change):
    model_dict["implementation"]=change['new']


@GPU_view.capture(clear_output=True)
def handle_GPU_checkbox(change):
    if change["new"] == True:
        model_dict["use_GPU"]=True
        print("Using the GPU")
    else:
        model_dict["use_GPU"]=False
        print("Not using the GPU")

        
@GPU_view.capture(clear_output=True)
def handle_CRF_checkbox(change):
    if change["new"] == True:
        model_dict["use_CRF"]=True
        print("Using CRF post-processing")
    else:
        model_dict["use_CRF"]=False
        print("Not using CRF post-processing")
        

@model_view.capture(clear_output=True)
def use_data_button_clicked(button):
    # Use the data folder as the input for segmentation
    print("Loading in the jpgs from the data directory")
    # Copy the jpgs from data to a new folder called segmentation_data_[datetime]
    if 'MNDWI' in model_dropdown.value:
        sample_direc =r'C:\1_USGS\CoastSeg\repos\2_CoastSeg\CoastSeg_fork\Seg2Map\MNDWI_outputs\MNDWI_ouputs_2022-07-21__07_hr_57_min14'
#         sample_direc = file_functions.get_jpgs_from_data('MNDWI')
#         RGB_path=sample_direc+os.sep+'RGB'
#         NIR_path=sample_direc+os.sep+'NIR'
#         sample_direc = coastseg_map.RGB_to_MNDWI(RGB_path,NIR_path,sample_direc)
        model_dict["sample_direc"]=sample_direc
    else:
        sample_direc = file_functions.get_jpgs_from_data('RGB')
        jpgs = glob.glob1(sample_direc + os.sep, "*jpg")
        if jpgs == []:
            # raise Exception("\nThe directory contains no jpgs! Please select a directory with jpgs.")
            print("\nERROR!\nThe directory contains no jpgs! Please select a directory with jpgs.")
        elif jpgs != []:
            model_dict["sample_direc"]=sample_direc
    print(f"\nContents of the data directory saved in {sample_direc}")


@model_view.capture(clear_output=True)
def use_select_images_button_clicked(button):
    # Prompt the user to select a directory of images                              
    root.withdraw()                                        # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
    # path to data directory containing data downloaded using coastsat
    data_path = os.path.join(os.getcwd(),'data')
    if os.path.exists(data_path):
        data_path = os.path.join(os.getcwd(),'data')
    else: 
        data_path = os.getcwd()
    root.filename =  filedialog.askdirectory(initialdir = data_path,title = "Select directory of images",)
    # Save the filename as an attribute of the button
    if root.filename:
        sample_direc= root.filename
        print(f"The images in the folder will be segmented :\n{sample_direc} ")
        jpgs = glob.glob1(sample_direc + os.sep, "*jpg")
        if jpgs == []:
            # raise Exception("\nThe directory contains no jpgs! Please select a directory with jpgs.")
            print("\nERROR!\nThe directory contains no jpgs! Please select a directory with jpgs.")
        elif jpgs != []:
            model_dict["sample_direc"]=sample_direc
    else:
        # raise Exception("You must select a valid directory first!")
        print("You must select a valid directory first!")

@run_model_view.capture(clear_output=True)
def run_model_button_clicked(button):
    print("Called Run Model")
    if model_dict["sample_direc"] is None:
        print("You must click 'Use Data' or 'Select Images' First")
        return
    else:
        if model_dict["use_GPU"] == False:
            print("Not using the GPU")
            ## to use the CPU (not recommended):
            os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
        elif model_dict["use_GPU"] == True:
            print("Using the GPU")
            ##use the first available GPU
            os.environ['CUDA_VISIBLE_DEVICES'] = '0' #'1'
        if 'MNDWI' in model_dropdown.value:
            dataset='MNDWI'
        else:
            dataset='RGB'
        # Disable run and open results buttons while the model is running
        open_results_button.disabled = True
        run_model_button.disabled = True
        dataset_id=model_dict['model_type']
        # model_dict['implementation']=model_implementation.value
        model_choice=model_dict['implementation']
        zoo_model=zoo_model_module.Zoo_Model()
        # First download the specified model
        zoo_model.download_model(dataset,dataset_id)
        # Get weights as list
        Ww=zoo_model.get_weights_list(model_choice)
        # Load the model from the config files
        model, model_list, config_files, model_types=zoo_model.get_model(Ww)
        metadatadict=zoo_model.get_metadatadict(Ww,config_files, model_types)
        # # Compute the segmentation
        zoo_model.compute_segmentation(model_dict["sample_direc"], model_list, metadatadict,model_dict['use_CRF'])
        # Enable  run and open results buttons when model has exexcuted
        run_model_button.disabled = False
        open_results_button.disabled = False

        
@run_model_view.capture(clear_output=True)
def open_results_button_clicked(button):
    if model_dict["sample_direc"] is None:
        print("You must click 'Run Model' first")
    else:
        # Open file viewer using tkinter                          
        root.withdraw()                                        # Hide the main window.
        root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
        # path to directory containing model outputs
        model_results_path = os.path.abspath(model_dict["sample_direc"])
        if not os.path.exists(model_results_path):
             raise FileNotFoundError
        filedialog.askopenfiles(initialdir = model_results_path,title = "Open Results")
    
    
model_implementation.observe(handle_model_implementation,"value")
model_dropdown.observe(handle_model_type,"value")
GPU_checkbox.observe(handle_GPU_checkbox,"value")
CRF_checkbox.observe(handle_CRF_checkbox,"value")


run_model_button.on_click(run_model_button_clicked)
use_data_button.on_click(use_data_button_clicked)
use_select_images_button.on_click(use_select_images_button_clicked)
open_results_button.on_click(open_results_button_clicked)


line_widget=HTML(value="____________________________________________________")

instr_header=HTML(
    value="<h4>Click ONE of the following buttons:</h4>",layout=Layout(margin='0px 0px 0px 0px'))

instr_use_data=HTML(
    value="<b>1. Use Data Folder Button</b> \
        <br> - When CoastSat downloads imagery it created a folder called 'data'in the CoastSeg directory.\
            The jpgs within the 'data' folder will be copied to another folder with a name such as\
            <span style=\"background-color:LightGray;color: black;\">segmentation_data_2022-07-07__10_hr_04_min58 </span>\
            (the date and time will be the current date and time) <br> \
           The model will be applied to this folder and the model outputs will be generated within a subdirectory \
            called 'out'",layout=Layout(margin='0px 0px 0px 20px'))

instr_select_images=HTML(
    value="<b>2. Select Images Button</b> \
        <br> - This will open a pop up window where the folder containing the jpgs can be selected.<br>\
             - The model will be applied to this folder and the model outputs will be generated within a subdirectory\
             called 'out'<br>\
      - <span style=\"background-color:yellow;color: black;\">WARNING :</span> You will not be able to see the files within the folder you select.<br>\
    ",layout=Layout(margin='0px 0px 0px 20px'))

instr_run_model=HTML(
    value="<b>3. Run Model Button</b> \
        <br> - Make sure to click Select Images Button or Use Data Button.<br>\
             - The model will be applied to the selected folder and the model outputs will be generated within a subdirectory\
             called 'out'<br>\
      - <span style=\"background-color:yellow;color: black;\">WARNING :</span> You should not run multiple models on the same folder. Otherwise not all the model outputs\
      will be saved to the folder.<br>\
    ",layout=Layout(margin='0px 0px 0px 20px'))

model_choices_box=HBox([model_dropdown, model_implementation])
checkboxes = HBox([GPU_checkbox,CRF_checkbox])
instr_vbox=VBox([instr_header, line_widget, instr_use_data, instr_select_images, instr_run_model])

display(checkboxes, GPU_view,model_choices_box,instr_vbox,use_data_button,use_select_images_button,line_widget, model_view,run_model_button,run_model_view,open_results_button)

Output()

Button(description='Use Data Button', style=ButtonStyle(button_color='#69add1'))

Button(description='Select Your Images', style=ButtonStyle(button_color='#69add1'))

HTML(value='____________________________________________________')

Output(layout=Layout(border='1px solid black'))

Button(description='Run Model', style=ButtonStyle(button_color='#ae3cf0'))

Output(layout=Layout(border='1px solid black'))

Button(description='Open Model Results', style=ButtonStyle(button_color='#69add1'))